# 📄 Hoja de Trabajo 2
Integrantes
- Diego Alexander Hernández Silvestre - 21270
- Mario Antonio Guerra Morales - 21008
- Linda Inés Jiménez Vides 21169

## 💻 Ejercicio 1 
### Preparación del conjunto de datos 📈

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.nn.functional as F

In [48]:
# Cargar el conjunto de datos de Iris
iris = load_iris()
X = iris.data  # Características
y = iris.target  # Etiquetas

df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Dividir el conjunto de datos en entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Mostrar los tamaños de los conjuntos resultantes
print(f"Conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Conjunto de validación: {X_test.shape[0]} muestras")
df_iris.head()


Conjunto de entrenamiento: 105 muestras
Conjunto de validación: 45 muestras


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### 🧠 Arquitectura del modelo

In [49]:
# Red neuronal feedforward
class SimpleFeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleFeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)  # Capa de entrada
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)  # Capa oculta
        self.fc3 = nn.Linear(hidden_size2, output_size)  # Capa de salida
        
        self.relu = nn.ReLU()  # Función de activación ReLU
        self.softmax = nn.Softmax(dim=1)  # Función de activación Softmax para la capa de salida

    def forward(self, x):
        # Pasar los datos a través de la red
        x = self.relu(self.fc1(x))  
        x = self.relu(self.fc2(x))  
        x = self.softmax(self.fc3(x)) 
        return x

# Parámetros de la red
input_size = 4  # Número de características de entrada 
hidden_size1 = 10  # Número de neuronas en la primera capa oculta
hidden_size2 = 8  # Número de neuronas en la segunda capa oculta
output_size = 3  # Número de clases de salida 

# Instancia de la red neuronal
model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size)

# Mostrar la estructura de la red
print(model)


SimpleFeedforwardNN(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)


### 📉 Funciones de perdida

In [50]:
# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [51]:
# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertir a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# Crear DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [52]:
def loss_functions():
    # Definir las funciones de pérdida a comparar
    loss_functions = [
        ('CrossEntropy', nn.CrossEntropyLoss()),
        ('MSE', nn.MSELoss()),
        ('NLL', nn.NLLLoss())
    ]
    
    results = []
    
    # Iterar sobre cada función de pérdida
    for loss_name, criterion in loss_functions:
        # Inicializar el modelo y el optimizador
        model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        
        train_losses = []
        test_losses = []
        
        # Entrenamiento durante 100 épocas
        for epoch in range(100):
            model.train()  # Poner el modelo en modo entrenamiento
            outputs = model(X_train_tensor.to(device))  # Hacer una pasada hacia adelante
            
            # Calcular la pérdida según la función de pérdida
            if loss_name == 'CrossEntropy':
                loss = criterion(outputs, y_train_tensor.to(device))
            elif loss_name == 'NLL':
                log_probs = nn.functional.log_softmax(outputs, dim=1)
                loss = criterion(log_probs, y_train_tensor.to(device))
            else:
                # 'MSE' usa etiquetas one-hot
                loss = criterion(outputs, nn.functional.one_hot(y_train_tensor, num_classes=output_size).float().to(device))
            
            # Actualizar los gradientes
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            
            # Evaluar el modelo
            model.eval()  # Poner el modelo en modo evaluación
            with torch.no_grad():
                test_outputs = model(X_test_tensor.to(device))  # Hacer una pasada hacia adelante en el conjunto de prueba
                
                # Calcular la pérdida en el conjunto de prueba
                if loss_name == 'CrossEntropy':
                    test_loss = criterion(test_outputs, y_test_tensor.to(device))
                elif loss_name == 'NLL':
                    log_probs = nn.functional.log_softmax(test_outputs, dim=1)
                    test_loss = criterion(log_probs, y_test_tensor.to(device))
                else:
                    # 'MSE' usa etiquetas one-hot
                    test_loss = criterion(test_outputs, nn.functional.one_hot(y_test_tensor, num_classes=output_size).float().to(device))
                
                test_losses.append(test_loss.item())
        
        # Calcular la precisión
        model.eval()  # Poner el modelo en modo evaluación
        with torch.no_grad():
            test_outputs = model(X_test_tensor.to(device))
            _, predicted = torch.max(test_outputs.data, 1)  # Obtener las predicciones de clase
            accuracy = (predicted == y_test_tensor.to(device)).sum().item() / y_test_tensor.size(0)  # Calcular la precisión
        
        # Almacenar los resultados
        results.append((loss_name, train_losses[-1], test_losses[-1], accuracy))
    
    # Imprimir los resultados
    for loss_name, train_loss, test_loss, accuracy in results:
        print(f"Loss Function: {loss_name}")
        print(f"Final Training Loss: {train_loss:.4f}")
        print(f"Final Test Loss: {test_loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print()

loss_functions()

Loss Function: CrossEntropy
Final Training Loss: 0.5754
Final Test Loss: 0.5553
Accuracy: 1.0000

Loss Function: MSE
Final Training Loss: 0.0102
Final Test Loss: 0.0063
Accuracy: 1.0000

Loss Function: NLL
Final Training Loss: 0.5840
Final Test Loss: 0.5670
Accuracy: 1.0000



### 📊 Técnicas de Regularización

In [53]:
def regularization():
    # Tipos de regularización a evaluar
    reg_types = ['None', 'L1', 'L2', 'Dropout']
    results = []
    
    # Iterar sobre cada tipo de regularización
    for reg_type in reg_types:
        # Inicializar el modelo para cada tipo de regularización
        model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size).to(device)
        
        # Aplicar la regularización según el tipo
        if reg_type == 'Dropout':
            model.fc1 = nn.Sequential(
                model.fc1,
                nn.Dropout(0.5)  # Aplicar Dropout con probabilidad del 50%
            )
            model.fc2 = nn.Sequential(
                model.fc2,
                nn.Dropout(0.5)  # Aplicar Dropout con probabilidad del 50%
            )
        
        # Definir la función de pérdida y el optimizador
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        
        # Entrenamiento durante 100 épocas
        for epoch in range(100):
            model.train()  # Poner el modelo en modo entrenamiento
            outputs = model(X_train_tensor.to(device))  # Hacer una pasada hacia adelante
            
            # Calcular la pérdida
            loss = criterion(outputs, y_train_tensor.to(device))
            
            # Aplicar regularización L1 o L2 si corresponde
            if reg_type == 'L1':
                l1_lambda = 0.01  # Hiperparámetro de regularización L1
                l1_norm = sum(p.abs().sum() for p in model.parameters())  # Calcular la norma L1
                loss = loss + l1_lambda * l1_norm  # Agregar la regularización L1 a la pérdida
            elif reg_type == 'L2':
                l2_lambda = 0.01  # Hiperparámetro de regularización L2
                l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())  # Calcular la norma L2
                loss = loss + l2_lambda * l2_norm  # Agregar la regularización L2 a la pérdida
            
            # Actualizar los gradientes
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Evaluar el modelo
        model.eval()  # Poner el modelo en modo evaluación
        with torch.no_grad():
            test_outputs = model(X_test_tensor.to(device))  # Hacer una pasada hacia adelante en el conjunto de prueba
            _, predicted = torch.max(test_outputs.data, 1)  # Obtener las predicciones de clase
            accuracy = (predicted == y_test_tensor.to(device)).sum().item() / y_test_tensor.size(0)  # Calcular la precisión
            test_loss = criterion(test_outputs, y_test_tensor.to(device))  # Calcular la pérdida en el conjunto de prueba
        
        # Almacenar los resultados
        results.append((reg_type, test_loss.item(), accuracy))
    
    # Imprimir los resultados para cada tipo de regularización
    for reg_type, test_loss, accuracy in results:
        print(f"Regularization: {reg_type}")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print()

regularization()

Regularization: None
Test Loss: 0.5584
Accuracy: 1.0000

Regularization: L1
Test Loss: 0.8194
Accuracy: 0.7111

Regularization: L2
Test Loss: 0.6095
Accuracy: 1.0000

Regularization: Dropout
Test Loss: 0.5988
Accuracy: 1.0000



### 📈 Algoritmos de Optimización

In [ ]:
# Task 5

### 🧪 Experimentación y Análisis

In [ ]:
# Task 6

### 🗣️ Discusión

Task 7

## 📚 Ejercicio 2 - Attention is all you need

1. **¿Cuál es la principal innovación de la arquitectura Transformer?**
2. **¿Cómo funciona el mecanismo de atención del scaled dot-product?**
3. **¿Por qué se utiliza la atención de múltiples cabezales en Transformer?**
4. **¿Cómo se incorporan los positional encodings en el modelo Transformer?**
5. **¿Cuáles son algunas aplicaciones de la arquitectura Transformer más allá de la machine translation?**